## Planck Maps


In [2]:
import numpy as np
import shutil, os, string
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable
from astropy.table import Table
from astropy.coordinates import Angle
import astropy.units as u
import pyregion


fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/data/MFS/FornaxA/Planck/maps/'
outDir = '/home/maccagni/Projects/MFS/FornaxA/contData/inIms/'
tmpDir = rootDir+'reproject_outputs/'
imDir = rootDir+'composite_maps/'
dustDir = rootDir+'../dust/'
ffDir = rootDir+'../ff/'


inTabs = outDir+'../inTabs/'

#fint.rootdir = rootDir
#syn.rootdir = rootDir
#pb.rootdir = rootDir


tableName = inTabs+'planckTable.tbl'
tableFileNames= ascii.read(tableName,format='csv')
inputmap =  tableFileNames.columns[0]

#inputmap = table['Filename']
#outputmap = table['OutName']
#freq = table['Freq']

frList = [30000.0,44000.0,70000.0,100000.0,143000.0,217000.0,353000.0,545000.0,857000.0]
print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


	+---------+
	 Inputs loaded
	+---------+


We measure the noise of images from 24 frequency intervals, between 84 MHz and 217 GHz.

In [3]:
#-------------------------------------------------#
# Regrid Planck maps using montage
#-------------------------------------------------#
import montage_wrapper as montage

width = 6.0 #degrees
px_size = 60.0 #arcseconds
centre = [50.673830, -37.208230]

NAXIS1 = int(width/(px_size/3600.))
NAXIS2 = int(NAXIS1)
CRPIX1 = NAXIS1/2.
CRPIX2 = NAXIS2/2.
CDELT1 = -px_size/3600.
CDELT2 = px_size/3600.
CRVAL1 = centre[0]
CRVAL2 = centre[1]


hdu = fits.PrimaryHDU()
hdr = hdu.header
hdr['NAXIS'] = 2
hdr['NAXIS1']  = NAXIS1
hdr['NAXIS2']  = NAXIS2
hdr['CTYPE1']  = 'RA---TAN'
hdr['CTYPE2']  = 'DEC--TAN'
hdr['CRVAL1']  = CRVAL1
hdr['CRVAL2']  = CRVAL2
hdr['CRPIX1']  = CRPIX1
hdr['CRPIX2']  = CRPIX2
hdr['CDELT1']  = CDELT1
hdr['CDELT2']  = CDELT2
hdr['EQUINOX']  = 2000.0
d = np.zeros([hdr['NAXIS2'],hdr['NAXIS1']])
fits.writeto(tmpDir+'tmp.fits',d,hdr,overwrite=True)

input_table = tableName

table = ascii.read(input_table)
inputmap = table['Filename']
outputmap = table['OutName']
freq = table['Freq']

headername = tmpDir+'tmp.hdr'
a = montage.mGetHdr(tmpDir+'tmp.fits',headername)

os.chdir(imDir)
for i in xrange(0,len(inputmap)):

        inmap = imDir+inputmap[i]
        print inmap

        outmap = '../reproject_outputs/'+outputmap[i]
        aaa = string.split(outmap, '.fits')
        outmap1=aaa[0]+'_tmp.fits'

        #command = 'HPXcvt '+'-C'+str(1)+' '+inmap+' '+outmap1
        command = 'HPXcvt -C0 -cg -r  '+inmap+' '+outmap1
        os.system(command)

        command = 'mProject '+outmap1+' '+outmap+' '+headername
        os.system(command)
os.remove(headername)
os.remove(tmpDir+'tmp.fits')
print '''\\t+---------+\\n\\t Reprojection done\\n\\t+---------+'''


/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/LFI_CompMap_Foregrounds-commander-030_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/LFI_CompMap_Foregrounds-commander-044_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/LFI_CompMap_Foregrounds-commander-070_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-100_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-143_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-217_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-353_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-545_R3.00.fits
/home/maccagni/data/MFS/FornaxA/Planck/maps/composite_maps/HFI_CompMap_Foregrounds-commander-857_R3.00.fits
\t+---------+\n\t Reprojecti

In [4]:
#-------------------------------------------------#
# Dust emission at each Planck Frequency
#-------------------------------------------------#
outnames = np.array(['OD_Fa_big.fits','SI_Fa_big.fits','T_Fa_big.fits'])

#infile = fits.open(dustdir+outnames[0])
#tau = infile[1].data
#inhead = infile[1].header       
#fits.writeto(dustdir+outnames[0],tau,inhead,overwrite=True)

#infile = fits.open(dustdir+outnames[1])
#si = infile[1].data
#inhead = infile[1].header       
#fits.writeto(dustdir+outnames[1],si,inhead,overwrite=True)

#infile = fits.open(dustdir+outnames[2])
#T = infile[1].data
#inhead = infile[1].header       
#fits.writeto(dustdir+outnames[2],T,inhead,overwrite=True)

frequencies = ['30','44','70','100','143','217','353','545','857']

fr =np.array([30.,44.,70.,100.,143.,217.,353., 545., 857.])*1e9 #Hz 
hp = 6.626070040e-34 #Jy/s
C = 2.99792458e8 #m/s
kb = 1.38064852e-23 #Jy/K
nu0 = 353e9
BB = 2.*hp/(C*C)


beams = [32.239, 27.005, 13.252, 9.651, 7.248, 4.990, 4.818, 4.682, 4.325]
dustnames=[]


for i in xrange(0,len(frequencies)):

    filename = dustDir+outnames[0]

    infile = fits.open(filename)
    tau = infile[0].data
    filename =   dustDir+outnames[1]
    infile = fits.open(filename)
    si = infile[0].data
    filename =   dustDir+outnames[2]
    infile = fits.open(filename)
    T = infile[0].data
    inhead = infile[0].header     

    if 'NAXIS3' in inhead:
        del inhead['NAXIS3']
    if 'NAXIS4' in inhead:
        del inhead['NAXIS4']  

#    pix_x = Angle(inhead['CDELT1'],unit=u.degree)
#    pix_y = Angle(inhead['CDELT2'],unit=u.degree)
#    pix_area =-pix_x.arcsecond*pix_y.arcsecond     

    hnu = hp*fr[i] 
    kT =np.multiply(T,kb)
    B_t = BB * np.power(fr[i],3) * np.divide(1,np.exp(np.divide(hnu,kT))-1)

    freq_ratio = np.power(fr[i]/nu0,si)
    tau_freq_ratio = np.multiply(tau,freq_ratio)

    dust = np.multiply(B_t,tau_freq_ratio) #Jy
    dust *=1e20 #MJy/sr

    #bmaj = Angle(beams[i],unit=u.arcmin)
    #bmin = Angle(beams[i],unit=u.arcmin)        
    #beam_area = 2*np.pi*bmaj.arcsecond/2.35482*bmin.arcsecond/2.35482
    #dust = np.multiply(dust,pix_area)    
    #arr = 2.350443e-5 # Jy/sr
    #dust =np.multiply(arr, dust) #Jy/asec2
    #dust = np.multiply(dust,beam_area/pix_area)    

    outfile = dustDir+frequencies[i]+'_dust_big.fits'
    dustnames.append(outfile)

    fits.writeto(outfile,dust,inhead,overwrite=True)

#outfile = dustoutdir+'353_dust_big.fits'  
#infile = fits.open(outfile)
#tau = infile[1].data
#print tau.shape
#inhead = infile[1].header  
#print inhead

#fits.writeto(outfile,tau,inhead,overwrite=True)
#dustnames.append(outfile)
#outfile = dustoutdir+'545_dust_big.fits'    
#infile = fits.open(outfile)
#tau = infile[1].data
#inhead = infile[1].header 
#print inhead

#fits.writeto(outfile,tau,inhead,overwrite=True)
#dustnames.append(outfile)
#outfile = dustoutdir+'857_dust_big.fits'    
#infile = fits.open(outfile)
#tau = infile[1].data
#print tau.shape
#inhead = infile[1].header  
#print inhead
#fits.writeto(outfile,tau,inhead,overwrite=True)
#dustnames.append(outfile)

print dustnames

print '''\\t+---------+\\n\\t Dust generated\\n\\t+---------+'''

MAP_EXT Original map extension                                                   [astropy.io.fits.card]


['/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/30_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/44_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/70_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/100_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/143_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/217_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/353_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/545_dust_big.fits', '/home/maccagni/data/MFS/FornaxA/Planck/maps/../dust/857_dust_big.fits']
\t+---------+\n\t Dust generated\n\t+---------+


In [5]:
#-------------------------------------------------#,
# Free free at each Planck Frequency,
#-------------------------------------------------#,
ff_dir = rootDir+'/../ff/'

EMname = ff_dir+'Fa_EM_big.fits'
Tname =  ff_dir+'Fa_T_mean_big.fits'

EMfile = fits.open(EMname)
EM = EMfile[1].data
EMheader = EMfile[1].header
EMfile.close()
Tfile = fits.open(Tname)
T = Tfile[1].data
Theader= Tfile[1].header
Tfile.close()
T4 = T/1e4
pi3 = np.sqrt(3)/np.pi

hp = 6.626070040e-34 #Jy/s
C = 2.99792458e8 #m/s
kb = 1.38064852e-23 #Jy/K

frequencies = ['30','44','70','100','143','217','353','545','857']
beams = [32.239, 27.005, 13.252, 9.651, 7.248, 4.990, 4.818, 4.682, 4.325]
fr =np.array([30.,44.,70.,100.,143.,217.,353.,545.,857.]) #GHz 

ffnames=[]

for i in xrange(0,len(frequencies)):

    gff = np.log(np.exp(5.960-pi3*np.log(fr[i]*np.power(T4,-3./2.)))+np.e)
    tau = 0.05468 * np.power(T,-3./2.)*np.power(fr[i],-2)*EM*gff  
    sff = T*(1-np.exp(-tau,dtype=float))#K  
    Iff = np.multiply(sff,2*kb/np.power(C/(fr[i]*1e9),2)) 
    Iff *= 1e20       
    outfile1 = ff_dir+frequencies[i]+'_ff_big.fits'
    
    fits.writeto(outfile1,Iff,Theader,overwrite=True)
    
    ffnames.append(outfile1)
    
print '''\\t+---------+\\n\\t Free Free generated\\n\\t+---------+'''
    


\t+---------+\n\t Free Free generated\n\t+---------+


In [6]:
#-------------------------------------------------#
# Convert to Jy/asec^2 and subtract Fg
#-------------------------------------------------#
T_cmb = 2.725 
C=2.99792458E10     #cm/s

frequencies = ['30','44','70','100','143','217','353','545','857']
frq =np.array([30.,44.,70.,100.,143.,217.,353.,545.,857.]) #GHz 
beam = [32.239, 27.005, 13.252, 9.651, 7.248, 4.990, 4.818, 4.682, 4.325]
print len(beam)
print frq.shape

print len(inputmap)
for i in xrange(0,len(inputmap)):
    
    freq = str(frq[i])
    beams = str(beam[i])
    
    
    inmap = rootDir+'/reproject_outputs/'+outputmap[i]
    
    aaa = string.split(outputmap[i], '.fits')
    outputmapName=aaa[0]
    outmap =  outDir+outputmapName+'_Fg_jybeam_bsub.fits'
    outmap_dusub = tmpDir+outputmapName+'_Fg_dusub.fits'
    outmap_unit = tmpDir+outputmapName+'_Fg_corrunit.fits'
    outmap_allsub = tmpDir+outputmapName+'_Fg_allsub.fits'

    infile = fits.open(inmap)
    array_coords = infile[0].data
    inhead = infile[0].header
    if 'NAXIS3' in inhead:
        del inhead['NAXIS3']
    if 'NAXIS4' in inhead:
        del inhead['NAXIS4']

    if frequencies[i] == '545' or frequencies[i] == '857':

        fits.writeto(outmap_unit,array_coords,inhead,overwrite=True)

        #subtract dust
        dustfile = dustDir+frequencies[i]+'_dust_big.fits'
        dust = fits.open(dustfile)
        dustdata = dust[0].data
        array_coords = np.subtract(array_coords,dustdata)                
        freefile=ff_dir+frequencies[i]+'_ff_big.fits'
        freename = fits.open(freefile)
        freedata = freename[0].data    
        array_coords = np.subtract(array_coords,freedata)                                                        
        fits.writeto(outmap_dusub,array_coords,inhead,overwrite=True)
        
        if str(frequencies[i]) == '545':
            #CIB
            array_coords -= 0.35
            #Zodiacal_light                                       
            array_coords += 0.04
        elif str(frequencies[i]) == '857':
            array_coords -= 0.64 
            array_coords += 0.12

        fits.writeto(outmap_allsub,array_coords,inhead,overwrite=True)

        #convert units    
        arr = 2.350443e-5 # Jy/sr
        array_coords =np.multiply(arr, array_coords) #Jy/asec2
    else:

        #Zodiacal_light                                           
        if str(frequencies[i]) == '100':
        #    print 'ciao'
            array_coords +=4.3e-7
        elif str(frequencies[i]) == '143':
            array_coords += 9.4e-7
        elif str(frequencies[i]) == '217':
            array_coords +=3.8e-6
        elif str(frequencies[i]) == '353':
            array_coords +=3.4e-5
        
        #convert units                                           
        sigma = np.divide(float(frq[i]),29.979)
        x = np.multiply(sigma,1.4388/T_cmb)
        sigmapow = np.power(sigma,3)
        xexp = np.exp(x)
        cmb = 1.e-12*sigmapow/(xexp-1)
        arr = cmb * x*xexp/(xexp-1)/T_cmb*1e4/C/1e-20 # MJy/sr
        array_coords =np.multiply(arr, array_coords)
        fits.writeto(outmap_unit,array_coords,inhead,overwrite=True)

        #subtract dust
        dustfile = dustDir+frequencies[i]+'_dust_big.fits'
        dust = fits.open(dustfile)
        dustdata = dust[0].data
        array_coords = np.subtract(array_coords,dustdata)       
        fits.writeto(outmap_dusub,array_coords,inhead,overwrite=True)

        #subtract ff                                        
        freefile=ff_dir+frequencies[i]+'_ff_big.fits'
        freename = fits.open(freefile)
        freedata = freename[0].data
        array_coords = np.subtract(array_coords,freedata)

        #subtract CIB
        if str(frequencies[i]) == '100':
            array_coords -=0.003
        elif str(frequencies[i]) == '143':
            array_coords -= 0.0079
        elif str(frequencies[i]) == '217':
            array_coords -= 0.033
        elif str(frequencies[i]) == '353':
            array_coords -= 0.13

        fits.writeto(outmap_allsub,array_coords,inhead,overwrite=True)

        arr = 2.350443e-5 # Jy/sr
        array_coords =np.multiply(arr, array_coords) #Jy/asec2

        
            
        pix_x = Angle(inhead['CDELT1'],unit=u.degree)
        pix_y = Angle(inhead['CDELT2'],unit=u.degree)
        pix_area =-pix_x.arcsecond*pix_y.arcsecond            
        bmaj = Angle(beam[i],unit=u.arcmin)
        bmin = Angle(beam[i],unit=u.arcmin)        
        beam_area = 2*np.pi*bmaj.arcsecond/2.35482*bmin.arcsecond/2.35482
        number_beam_pix = beam_area

        array_coords = np.multiply(array_coords,number_beam_pix)

       # measure mean background and noise
        in_reg = outDir+'/regions/ToT.reg'
        hdr = fits.Header()
        hdr['SIMPLE']  = 'T'
        hdr['BITPIX']  = -32
        hdr['NAXIS']   = 2
        hdr['CUNIT1']  = 'deg'
        hdr['CUNIT2']  = 'deg'
        hdr['EPOCH'] = 2000.
        hdr['NAXIS1']  = inhead['NAXIS1']
        hdr['NAXIS2']  = inhead['NAXIS1']
        hdr['CRPIX1']  = inhead['CRPIX1']                                              
        hdr['CDELT1']  = inhead['CDELT1']                                              
        hdr['CRVAL1']  = inhead['CRVAL1']                                             
        hdr['CTYPE1']  = 'RA'                                                    
        hdr['CRPIX2']  = inhead['CRPIX2']                                             
        hdr['CDELT2']  = inhead['CDELT2']                                              
        hdr['CRVAL2']  = inhead['CRVAL2']                                               
        hdr['CTYPE2']  = 'DEC'
        shape = (inhead['NAXIS2'], inhead['NAXIS1'])    
        r = pyregion.open(in_reg).as_imagecoord(hdr)
        m = r.get_mask(shape=shape)
        mask_back=np.copy(array_coords)
        mask_back[np.where(m==True)]  = np.nan
        back = np.nanmean(mask_back)     
        noise = np.nanstd(mask_back)            
        
        
       #subtract mean background    
        #if float(frq[i])<100 :
        #    array_coords-= back

        fits.writeto(outmap,array_coords,inhead,overwrite=True)

print '''\\t+---------+\\n\\t Coordinate transformation done\\n\\t+---------+'''

9
(9,)
9
\t+---------+\n\t Coordinate transformation done\n\t+---------+
